In [1]:
from IPython.display import display
from IPython.display import HTML
import IPython.core.display as di # Example: di.display_html('<h3>%s:</h3>' % str, raw=True)

# This line will hide code by default when the notebook is exported as HTML
# di.display_html('<script>jQuery(function() {if (jQuery("body.notebook_app").length == 0) { jQuery(".input_area").toggle(); jQuery(".prompt").toggle();}});</script>', raw=True)

display(HTML("<style>.container { width:100% !important; }</style>"))

import hail as hl
hl.init()

Running on Apache Spark version 2.4.1
SparkUI available at http://633fc3f91d0f:4040
Welcome to
     __  __     <>__
    / /_/ /__  __/ /
   / __  / _ `/ / /
  /_/ /_/\_,_/_/_/   version 0.2.29-cf730c8fc8f6
LOGGING: writing to /hail/large_vcf_filtering_annotation/hail-20191220-0816-0.2.29-cf730c8fc8f6.log


In [2]:
mt = hl.read_matrix_table('step2_output.mt')

In [4]:
gcov = hl.read_table('/hail/gnomad/gnomad.genomes.r3.0.coverage.ht')

In [6]:
gcov = gcov.select(gcov.over_1)

In [7]:
mt = mt.annotate_rows(cov_v3 = gcov[mt.locus])

In [9]:
mt = mt.filter_rows(mt.cov_v3.over_1 > 0.9)

In [10]:
gnmd = hl.read_table('/hail/gnomad/gnomad.genomes.r3.0.sites.ht')

In [11]:
gnmd = gnmd.select(v3_nfe = gnmd.freq[2], 
            intergenic_consequences = gnmd.vep.intergenic_consequences,
            most_severe_consequence = gnmd.vep.most_severe_consequence,
            motif_feature_consequences = gnmd.vep.motif_feature_consequences,
            regulatory_feature_consequences = gnmd.vep.regulatory_feature_consequences,
            transcript_consequences = gnmd.vep.transcript_consequences,
            variant_class = gnmd.vep.variant_class,
            rsid = gnmd.rsid)

In [12]:
mt = mt.annotate_rows(gnomad_v3 = gnmd[mt.row_key])

In [14]:
mt.checkpoint('step3_output.mt')

2019-12-20 09:11:13 Hail: INFO: wrote matrix table with 7615409 rows and 151 columns in 591 partitions to step3_output.mt
